# This notebook preprocesses a csv file containing texts from newspapers for a word embedding model

In [ ]:
## Import packages
#!pip install --upgrade pandas
import pandas as pd
from google.colab import drive

!pip install ftfy
import ftfy
from ftfy import fix_and_explain, apply_plan, fix_text

import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download('stopwords')

from itertools import chain

     |████████████████████████████████| 71kB 3.5MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.1-cp37-none-any.whl size=41573 sha256=3c11fcbf750afb63bf6e61b9889d97eb2b109b9991ce0e0e1e6e2b3f92c28c67
  Stored in directory: /root/.cache/pip/wheels/ae/73/c7/9056e14b04919e5c262fe80b54133b1a88d73683d05d7ac65c
Successfully built ftfy
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
## Mount drive
drive.mount('/content/drive')

## Load data
data_path = 'path to data folder'
file_name = '1792_1870' # Change here to preprocess a different time period
extension = '.zip'
df = pd.read_csv(data_path+file_name+extension, encoding='latin1', parse_dates=['date'])

Mounted at /content/drive


In [ ]:
# View data
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200984 entries, 0 to 200983
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  200984 non-null  int64         
 1   index       200984 non-null  int64         
 2   identifier  200984 non-null  object        
 3   type        200984 non-null  object        
 4   title       200951 non-null  object        
 5   papertitle  200984 non-null  object        
 6   date        200984 non-null  datetime64[ns]
 7   text        200983 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 12.3+ MB
None


,Unnamed: 0,index,identifier,type,title,papertitle,date,text
0,0,0,http://resolver.kb.nl/resolve?urn=MMKB08:00008...,artikel,INGEZONDEN. Aan de Redactie der Arnhemsche Cou...,C.A. Thieme,1861-10-16,In onze stadscourant las ik dezer dagen het vo...
1,1,1,http://resolver.kb.nl/resolve?urn=ddd:01080548...,artikel,DE VROUW.,L.E. Bosch & Zoon,1869-09-20,"â De Rott. CL, die op het punt der vrouw zee..."
2,2,2,http://resolver.kb.nl/resolve?urn=MMKB08:00008...,artikel,NOG IETS OVER DE MEDISCHE VOLMAKINGSSCHOOL TE ...,C.A. Thieme,1867-03-01,Is het plan der medische volmakingsschool niet...
3,3,3,http://resolver.kb.nl/resolve?urn=MMKB08:00008...,artikel,"OVER DE ZOOGENAAMDE âEMANCIPATIE"" DER VROUW....",C.A. Thieme,1868-08-29,"Vergun aan eenen uwer meest bestendige lezers,..."
4,4,4,http://resolver.kb.nl/resolve?urn=ddd:01078643...,artikel,Javaansche Amazonen.,Assen : van Gorcum,1845-09-26,In de Revue de Paris leest men: Â»'t Kijk van ...
5,5,5,http://resolver.kb.nl/resolve?urn=MMKB08:00011...,advertentie,Advertentie,J.H. Molenbroek,1869-07-18,"obstetriiiee , en, dames hebben er geen zittin..."
6,6,6,http://resolver.kb.nl/resolve?urn=ddd:01027554...,advertentie,Advertentie,Wed. J. Abrahams & Zoon,1869-01-07,GEEN VROUWEN-EMANCIPATIE? EEN WOORD AAN Mevrou...
7,7,7,http://resolver.kb.nl/resolve?urn=ddd:01027556...,advertentie,Advertentie,Wed. J. Abrahams & Zoon,1869-02-06,"Bij den Uitgever P. J. MILBORN te Middelburg, ..."
8,8,8,http://resolver.kb.nl/resolve?urn=ddd:01110094...,artikel,ENGELAND.,Zadelhoff & Fabritius,1862-10-18,Op 27 Julij 11. slond zekere John Aylesfort vo...
9,9,9,http://resolver.kb.nl/resolve?urn=MMKB08:00008...,artikel,Bij deze Courant behoort een Bijvoegsel. ARNHE...,C.A. Thieme,1869-11-22,"gen 'klaagt over den invloed, dien de franjche..."


In [ ]:
## remove duplicate entries based on the text column
df = df.drop_duplicates(subset=['text'])

## remove unnecessary columns
df = df.drop(['Unnamed: 0', 'identifier'], axis=1, errors='ignore')

## reset the index
df = df.reset_index()
print(df.info())

## check for NA in text & drop
print("NA values in text:",df['text'].isna().any())
print(type(df))
df = df.dropna(axis=0, how='any', subset=['text'])
print("NA values in text:",df['text'].isna().any())
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200984 entries, 0 to 200983
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   level_0     200984 non-null  int64         
 1   index       200984 non-null  int64         
 2   type        200984 non-null  object        
 3   title       200951 non-null  object        
 4   papertitle  200984 non-null  object        
 5   date        200984 non-null  datetime64[ns]
 6   text        200983 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 10.7+ MB
None
NA values in text: True
<class 'pandas.core.frame.DataFrame'>
NA values in text: False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200983 entries, 0 to 200983
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   level_0     200983 non-null  int64         
 1   index       200983 non-nu

In [ ]:
# View one text
df['text'][10]

'Onder bovenstaand opschrift bevat de Arnli. Ct. het volgende ingezonden stuk:. â\x80¢ Â« Vergun aan eeneu ; uwer meest, bestendige lezers , mijnheer de redacteur, u de redenen \'op te geven , Waarom hij\'; niet. kan instemmen met hetgeen gij ia "uw nummer, vau 15 Augustus geschreven liebt over de Â» emancipatie der vrouw" Â» In de perste plaats , is dat grootsch klinkende woord emancipatie hier wel goed , gekozen ? ~ Ik geloof het niet. Al kennen wij ome vrouwen geeue politieke regten toe, al roepen wij , haar Â«iet in : de raadzaal en in de pleitzaal, â\x80\x9e wij beschouwen en behandelen haar daarom niÃ©t â\x96\xa0 als slaven, lijfeigenen, minderjarigen of onnouzelen. Do â\x80¢ invloed der vrouw is in onze maatschappij zeer sterkt niet enkel iu het huiselijke, raam\' ook in het publieke , in het gansche maatschappelijk leven.,-, eu naar mijne overtuiging, is die invloed der vrouw op en in onze maatschappij juist daarom zoo sterk, omdat hij een moreele, d. i. een .middellijk werkend

In [ ]:
## Fix encoding issues
for index, row in df.iterrows():
  text = row['text']
  text_fixed = fix_text(text)
  df.loc[index, 'text'] = text_fixed

# Check one text
df['text'][10]

'Onder bovenstaand opschrift bevat de Arnli. Ct. het volgende ingezonden stuk:. • « Vergun aan eeneu ; uwer meest, bestendige lezers , mijnheer de redacteur, u de redenen \'op te geven , Waarom hij\'; niet. kan instemmen met hetgeen gij ia "uw nummer, vau 15 Augustus geschreven liebt over de » emancipatie der vrouw" » In de perste plaats , is dat grootsch klinkende woord emancipatie hier wel goed , gekozen ? ~ Ik geloof het niet. Al kennen wij ome vrouwen geeue politieke regten toe, al roepen wij , haar «iet in : de raadzaal en in de pleitzaal, " wij beschouwen en behandelen haar daarom niét ■ als slaven, lijfeigenen, minderjarigen of onnouzelen. Do • invloed der vrouw is in onze maatschappij zeer sterkt niet enkel iu het huiselijke, raam\' ook in het publieke , in het gansche maatschappelijk leven.,-, eu naar mijne overtuiging, is die invloed der vrouw op en in onze maatschappij juist daarom zoo sterk, omdat hij een moreele, d. i. een .middellijk werkende , invloed is. ïxDe.eisch der 

In [ ]:
## Tokenization white-space

# Tokenize to sentences
df['sen_tokenized'] = df.apply(lambda row: nltk.sent_tokenize(row['text'], language="dutch"), axis=1)

# Tokenize to words with white-space
tokenizer = RegexpTokenizer(r'\w+')
tokenized = []
for index, row in df.iterrows():
  sent = row['sen_tokenized']
  tokens = [tokenizer.tokenize(sentence) for sentence in sent]
  tokenized.append(tokens)

df['tokenized'] = tokenized

# Make a new column with the length of each tokenized article
df['length'] = df.apply(lambda row: sum([len(x) for x in row['tokenized']]), axis=1)

# Check the total amount of tokens
print("Total tokens data-set: ",sum(df['length']))

# Check one text
df['tokenized'][10]

Total tokens data-set:  118449415


[['Onder', 'bovenstaand', 'opschrift', 'bevat', 'de', 'Arnli'],
 ['Ct'],
 ['het', 'volgende', 'ingezonden', 'stuk'],
 ['Vergun',
  'aan',
  'eeneu',
  'uwer',
  'meest',
  'bestendige',
  'lezers',
  'mijnheer',
  'de',
  'redacteur',
  'u',
  'de',
  'redenen',
  'op',
  'te',
  'geven',
  'Waarom',
  'hij',
  'niet'],
 ['kan',
  'instemmen',
  'met',
  'hetgeen',
  'gij',
  'ia',
  'uw',
  'nummer',
  'vau',
  '15',
  'Augustus',
  'geschreven',
  'liebt',
  'over',
  'de',
  'emancipatie',
  'der',
  'vrouw',
  'In',
  'de',
  'perste',
  'plaats',
  'is',
  'dat',
  'grootsch',
  'klinkende',
  'woord',
  'emancipatie',
  'hier',
  'wel',
  'goed',
  'gekozen'],
 ['Ik', 'geloof', 'het', 'niet'],
 ['Al',
  'kennen',
  'wij',
  'ome',
  'vrouwen',
  'geeue',
  'politieke',
  'regten',
  'toe',
  'al',
  'roepen',
  'wij',
  'haar',
  'iet',
  'in',
  'de',
  'raadzaal',
  'en',
  'in',
  'de',
  'pleitzaal',
  'wij',
  'beschouwen',
  'en',
  'behandelen',
  'haar',
  'daarom',
  'ni

In [ ]:
## Remove punctuation - keep only alphabetical characters

no_punct = []
for index, row in df.iterrows():
  text = list(chain.from_iterable(row['tokenized'])) #chain.from_iterable unlists the sentence-word sublist structure
  npun = [word for word in text if word.isalpha()]
  no_punct.append(npun)

df['no_punct'] = no_punct

# Check one text  
df['no_punct'][10]

['Onder',
 'bovenstaand',
 'opschrift',
 'bevat',
 'de',
 'Arnli',
 'Ct',
 'het',
 'volgende',
 'ingezonden',
 'stuk',
 'Vergun',
 'aan',
 'eeneu',
 'uwer',
 'meest',
 'bestendige',
 'lezers',
 'mijnheer',
 'de',
 'redacteur',
 'u',
 'de',
 'redenen',
 'op',
 'te',
 'geven',
 'Waarom',
 'hij',
 'niet',
 'kan',
 'instemmen',
 'met',
 'hetgeen',
 'gij',
 'ia',
 'uw',
 'nummer',
 'vau',
 'Augustus',
 'geschreven',
 'liebt',
 'over',
 'de',
 'emancipatie',
 'der',
 'vrouw',
 'In',
 'de',
 'perste',
 'plaats',
 'is',
 'dat',
 'grootsch',
 'klinkende',
 'woord',
 'emancipatie',
 'hier',
 'wel',
 'goed',
 'gekozen',
 'Ik',
 'geloof',
 'het',
 'niet',
 'Al',
 'kennen',
 'wij',
 'ome',
 'vrouwen',
 'geeue',
 'politieke',
 'regten',
 'toe',
 'al',
 'roepen',
 'wij',
 'haar',
 'iet',
 'in',
 'de',
 'raadzaal',
 'en',
 'in',
 'de',
 'pleitzaal',
 'wij',
 'beschouwen',
 'en',
 'behandelen',
 'haar',
 'daarom',
 'niét',
 'als',
 'slaven',
 'lijfeigenen',
 'minderjarigen',
 'of',
 'onnouzelen',
 'Do'

In [ ]:
## Make lowercase
lowercase = []
for index, row in df.iterrows():
  text = row['no_punct']
  lc = [w.lower() for w in text]
  lowercase.append(lc)

df['lowercase'] = lowercase

# Check one text  
df['lowercase'][10]

['onder',
 'bovenstaand',
 'opschrift',
 'bevat',
 'de',
 'arnli',
 'ct',
 'het',
 'volgende',
 'ingezonden',
 'stuk',
 'vergun',
 'aan',
 'eeneu',
 'uwer',
 'meest',
 'bestendige',
 'lezers',
 'mijnheer',
 'de',
 'redacteur',
 'u',
 'de',
 'redenen',
 'op',
 'te',
 'geven',
 'waarom',
 'hij',
 'niet',
 'kan',
 'instemmen',
 'met',
 'hetgeen',
 'gij',
 'ia',
 'uw',
 'nummer',
 'vau',
 'augustus',
 'geschreven',
 'liebt',
 'over',
 'de',
 'emancipatie',
 'der',
 'vrouw',
 'in',
 'de',
 'perste',
 'plaats',
 'is',
 'dat',
 'grootsch',
 'klinkende',
 'woord',
 'emancipatie',
 'hier',
 'wel',
 'goed',
 'gekozen',
 'ik',
 'geloof',
 'het',
 'niet',
 'al',
 'kennen',
 'wij',
 'ome',
 'vrouwen',
 'geeue',
 'politieke',
 'regten',
 'toe',
 'al',
 'roepen',
 'wij',
 'haar',
 'iet',
 'in',
 'de',
 'raadzaal',
 'en',
 'in',
 'de',
 'pleitzaal',
 'wij',
 'beschouwen',
 'en',
 'behandelen',
 'haar',
 'daarom',
 'niét',
 'als',
 'slaven',
 'lijfeigenen',
 'minderjarigen',
 'of',
 'onnouzelen',
 'do'

In [ ]:
# Remove stopwords
stop_words = set(stopwords.words('dutch'))
# Keep personal pronouns
stop_words.remove("haar")
stop_words.remove("hem")
stop_words.remove("hij")
stop_words.remove("ze")
stop_words.remove("zij")
stop_words.remove("moet")
stop_words.remove("mijn")

sw = []
for index, row in df.iterrows():
  text = row['lowercase']
  swords = [w for w in text if not w in stop_words]
  sw.append(swords)

df['stopwords'] = sw

# Check one text  
df['stopwords'][10]

['bovenstaand',
 'opschrift',
 'bevat',
 'arnli',
 'ct',
 'volgende',
 'ingezonden',
 'stuk',
 'vergun',
 'eeneu',
 'uwer',
 'meest',
 'bestendige',
 'lezers',
 'mijnheer',
 'redacteur',
 'redenen',
 'geven',
 'waarom',
 'hij',
 'instemmen',
 'hetgeen',
 'gij',
 'ia',
 'nummer',
 'vau',
 'augustus',
 'geschreven',
 'liebt',
 'emancipatie',
 'vrouw',
 'perste',
 'plaats',
 'grootsch',
 'klinkende',
 'woord',
 'emancipatie',
 'wel',
 'goed',
 'gekozen',
 'geloof',
 'kennen',
 'wij',
 'ome',
 'vrouwen',
 'geeue',
 'politieke',
 'regten',
 'toe',
 'roepen',
 'wij',
 'haar',
 'iet',
 'raadzaal',
 'pleitzaal',
 'wij',
 'beschouwen',
 'behandelen',
 'haar',
 'daarom',
 'niét',
 'slaven',
 'lijfeigenen',
 'minderjarigen',
 'onnouzelen',
 'do',
 'invloed',
 'vrouw',
 'onze',
 'maatschappij',
 'zeer',
 'sterkt',
 'enkel',
 'iu',
 'huiselijke',
 'raam',
 'publieke',
 'gansche',
 'maatschappelijk',
 'leven',
 'eu',
 'mijne',
 'overtuiging',
 'invloed',
 'vrouw',
 'onze',
 'maatschappij',
 'juist',

In [ ]:
# Save relevant columns
df.info()
df_save = df.drop(["text", "sen_tokenized", "tokenized", "length", "no_punct","lowercase","index"], axis=1)
df_save.rename(columns = {'stopwords':'text'}, inplace = True)
df_save.info()

path = 'path to folder where to save data'+file_name+'_preprocessed.csv'
df_save.to_csv(path_or_buf=path, na_rep = 'NA', index_label=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200983 entries, 0 to 200983
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   level_0        200983 non-null  int64         
 1   index          200983 non-null  int64         
 2   type           200983 non-null  object        
 3   title          200950 non-null  object        
 4   papertitle     200983 non-null  object        
 5   date           200983 non-null  datetime64[ns]
 6   text           200983 non-null  object        
 7   sen_tokenized  200983 non-null  object        
 8   tokenized      200983 non-null  object        
 9   length         200983 non-null  int64         
 10  no_punct       200983 non-null  object        
 11  lowercase      200983 non-null  object        
 12  stopwords      200983 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(9)
memory usage: 26.5+ MB
<class 'pandas.core.frame.DataFrame'>
I